In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? t
Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import brain_clusters
import os
import pandas as pd
import numpy as np
import pandas as pd
import scipy.io as spio
import matplotlib.gridspec as gridspec
import cPickle as pickle
import scipy.stats as stats
import matplotlib.pyplot as plt

In [4]:
brain_clusters = reload(brain_clusters)

## calculate percent variance explained by PC1 for each dataset

In [3]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta'
df = pd.read_csv(os.path.join(SJdir, 'PCA', 'csvs_FINAL', 'mean_traces_all_subjs.csv'))

In [4]:
df.head()

,subj,task,elec,pattern,cluster,start_idx,end_idx,start_idx_resp,end_idx_resp,RTs,RTs_median,RTs_min,lats,lats_semi_static,lats_static,maxes,ROI
0,CP7,DecisionAud,11,SR,1,600,1381,57,500,2064.096774,1827,1253,89,89,689,136.943860,R
1,CP7,DecisionAud,12,S,2,600,983,0,0,2064.096774,1827,1253,100,100,700,133.239600,R
2,CP7,DecisionAud,14,S,2,600,1091,0,0,2064.096774,1827,1253,156,156,756,33.416164,R
3,CP7,DecisionAud,17,sustained,3,612,2230,-240,500,2064.096774,1827,1253,101,4112,4724,15.105470,R
4,CP7,DecisionAud,18,R,4,1454,3049,-344,500,2064.096774,1827,1253,723,723,2177,51.039216,R


In [12]:
pc1 = []
for i in df.groupby(['subj','task']):
    subj, task = i[0]

    filename = os.path.join(SJdir, 'PCA','Stats', 'Networks','unsmoothed','PCA_means','%s_%s_summary.txt' %(subj, task))
    list_of_lists = []
    with open(filename,'r') as f:
        for line in f:
            inner_list = [elt.strip() for elt in line.split(',')]
            # in alternative, if you need to use the file content as numbers
            # inner_list = [int(elt.strip()) for elt in line.split(',')]
            list_of_lists.append(inner_list)
        f.close()

    tmp = [x[0].split(' ') for x in list_of_lists if x[0].split(' ')[0:2] == ['Proportion', 'Var']][0]
    tmp = [t for t in tmp if len(t)>0]
    pc1.append(np.float(tmp[2]))

In [13]:
np.mean(pc1) #average percent variance accounted for by first PC in single trial PCA

0.20351351351351352

## plot PC thresholded elecs on brain

In [12]:
param = 'means'
PC = 'PC1'

In [4]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta'

In [ ]:
filename = os.path.join(SJdir,'PCA', 'Stats', 'single_electrode_windows_csvs', 'single_electrode_windows_withdesignation_EDITED.csv')
df_all = pd.read_csv(filename)

In [5]:
reconlist = os.path.join(SJdir, 'PCA','reconlist.csv')
reconlist = pd.read_csv(reconlist)

In [14]:
saveDir = os.path.join(SJdir, 'PCA', 'Stats', 'Networks', 'unsmoothed', 'PCA_'+param, 'networks', 'brains')
os.mkdir(saveDir)

subj_task = df_all[['subj', 'task']].drop_duplicates()

for s_t in subj_task.itertuples():
    subj, task = s_t[1:]
    
    
    xycoords = reconlist[reconlist.subj_task == '_'.join([subj, task])].xyfile.iloc[0]
    if isinstance(xycoords, float): #no explicit xycoord path
        xycoords = 'xycoords.p'
    
    reconfile = reconlist[reconlist.subj_task == '_'.join([subj, task])].reconfile.iloc[0]
    if len(reconfile)>0:
        recon_filepath = os.path.join(SJdir, 'Subjs', subj, reconfile)
    else:
        print 'skipping %s %s' %(subj, task)
        continue #skip this subj if no reconfile
        
    #open xycoords dictionary                                                                                           
    filename = os.path.join(SJdir, 'Subjs',subj, xycoords)
    with open(filename, 'r') as f:
        xycoords = pickle.load(f)
        f.close()

    xycoords = pd.DataFrame(np.array(xycoords.values()), columns=['x_2d', 'y_2d'], index=np.array(xycoords.keys())+1)
    
    #color stuff
    #import matplotlib
    color_list = ['crimson', 'skyblue','lime', 'goldenrod'] #duration, response, stimulus, sustained

    custom_cmap = matplotlib.colors.ListedColormap(color_list, name = 'custom_cmap')
    
    #get elecs of interest (for PC of interest)
    pc = df_thresh[(df_thresh.subj == subj) & (df_thresh.task == task) & (df_thresh.max_pc== PC)]
    
    weights = pc[['elec','pattern']]
    weights.columns = ['elec','group']
    weights = weights.set_index('elec')

    #create list of colors for scatter
    c = list()
    #create list of colors for scatter (dur = red, resp = blue, stim = green, sustain = yellow)
    c = list()
    u = np.unique(weights.group)
    for i in weights.group:
        if i == 'D':
            c.append(color_list[0])
        elif i == 'R':
            c.append(color_list[1])
        elif ((i == 'S') or (i == 'SR')) :
            c.append(color_list[2])
        else:
            c.append(color_list[3])
            
    #figure properties
    f, ax1 = plt.subplots(figsize = (10,10))
    ax1.set_title('%s %s %s' %(subj, task, PC))
    
    #plot recon
    brain_clusters.plot_xy_map(weights, locs = xycoords.loc[weights.index], ax = ax1, colors = c, cmap = custom_cmap, szmult=200, im_path = os.path.join(SJdir,'Subjs', subj, reconfile))    
    
    filename = os.path.join(saveDir, '_'.join([subj, task, PC+'.png']))
    plt.savefig(filename)
    plt.close()

## correlate PC1 and PC2 scores for different parameters with RTs

In [7]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta/'
filename = os.path.join(SJdir,'PCA', 'Stats', 'single_electrode_windows_csvs', 'single_electrode_windows_withdesignation_EDITED.csv')
df_all = pd.read_csv(filename)

subjs, tasks, elecs, Rs_PC1, Rs_PC2, ps_PC1, ps_PC2,feats  = [[] for i in range(8)]
R, pval = [dict() for i in range(2)]

features_to_consider = ['medians','means', 'stds']

for s_t in df_all.groupby(['subj','task']):
 
    subj, task = s_t[0]
    #if s_t[0]!=('CP7','DecisionAud'):
    #    continue

    for f in features_to_consider:

        filename = os.path.join(SJdir, 'PCA','Stats', 'Networks', 'unsmoothed', 'PCA_'+f, '_'.join([subj, task, 'scores.csv']))
        df = pd.read_csv(filename)
        filename = os.path.join(SJdir, 'PCA','Stats', 'outliers', 'for_PCA', 'unsmoothed', '_'.join([subj, task, f, 'RTs.csv']))
        df_RTs = pd.read_csv(filename)
        df_RTs = df_RTs.iloc[:,0]
        
        if not('PC2' in df.columns):
            df['PC2'] = np.nan

        for p in ['PC1','PC2']:
            R[p], pval[p] = stats.pearsonr(df[p], df_RTs)

        subjs.append(subj)
        tasks.append(task)
        Rs_PC1.append(R['PC1'])
        ps_PC1.append(pval['PC1'])
        Rs_PC2.append(R['PC2'])
        ps_PC2.append(pval['PC2'])
        feats.append(f)

        #plot
        fig, ax = plt.subplots(1,2, figsize= (10,3))
        colors = ['goldenrod','maroon']

        for i, p in enumerate(['PC1','PC2']):
            A = np.vstack([df_RTs,np.ones(len(df_RTs))]).T
            m,c = np.linalg.lstsq(A,np.array(df[p]))[0]

            ax[i].scatter(df_RTs, df[p], edgecolor = 'k', facecolor = colors[i], s = 75)
            ax[i].plot(df_RTs,df_RTs*m+c, color = 'k', linewidth = 4) 

            ax[i].set_title('R=%.2f, p= %.3f' %(R[p], pval[p]))
            ax[i].set_xlabel('RTs (ms)')
            ax[i].set_ylabel('%s score' %(p))
        fig.suptitle(f, size = 14)

        filename = os.path.join(SJdir, 'PCA', 'Stats', 'Correlations', 'unsmoothed', 'RTs_vs_PCs', '_'.join([subj, task, f]))
        plt.savefig(filename+'.png')
        plt.close()

df_all = pd.DataFrame({'subj':subjs, 'task':tasks, 'feature':feats, 'R_PC1':Rs_PC1, 'p_PC1':ps_PC1, 'R_PC2':Rs_PC2, 'p_PC2':ps_PC2})
df_all = df_all[['subj','task','feature','R_PC1','p_PC1','R_PC2','p_PC2']]

filename = os.path.join(SJdir, 'PCA','Stats', 'Correlations', 'unsmoothed', 'RTs_vs_PCs', 'RTs_vs_PCs_corr.csv')
df_all.to_csv(filename)

In [77]:
df_all

,subj,task,feature,R_PC1,p_PC1,R_PC2,p_PC2
0,CP7,DecisionAud,medians,-0.084817,0.352961,0.042555,0.641643
1,CP7,DecisionAud,maxes_rel,-0.089282,0.342679,0.089592,0.341000
